In [ ]:
import pandas as pd
import pymysql
from datetime import datetime
import pandas.io.sql as pds
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.font_manager import FontProperties

def get_datas(database, sql):
    cursor = database.cursor()  # 使用 cursor() 方法创建一个游标对象 curs
    try:
        cursor.execute(sql)
    except Exception as e:
        database.rollback()  # 发生错误时回滚
        print(str(e))
    else:
        database.commit()  # 事务提交
        datas = cursor.fetchall()
        cursor.close()
        return datas

In [ ]:
db = pymysql.connect("localhost", "root", "zgsxhm", "db_0221", use_unicode=True, charset="utf8")
sql_get_datas = 'select * from bytype '
datas = pds.read_sql(sql_get_datas, db)
font_set = FontProperties(fname="c:/myproject/pyExcel/cfets_reports/simfang.ttf", size=12)

In [ ]:
def by_bond_class(input_data, bond_class):
    data = input_data
    d1 = pd.pivot_table(data[(data.bond_class==bond_class)], index=["inst_type"], values=["vol"], 
                   columns=["side"], aggfunc=np.sum)
    x = np.arange(len(d1))
    # 绘制柱状图, 向上
    plt.xlabel("机构类型", fontproperties=font_set)

    plt.bar(x, np.array(d1.values[:,0]), facecolor='white', edgecolor='r')
    # 绘制柱状图, 向下
    plt.bar(x, -np.array(d1.values[:,1]), facecolor='white', edgecolor='g')
    plt.bar(x, np.array(d1.values[:,0])-np.array(d1.values[:,1]), facecolor='r', edgecolor='r')

    temp = zip(x, d1.values[:,1])
    temp2 = zip(x, d1.index)
    # 在柱状图上显示具体数值, ha水平对齐, va垂直对齐
    for x, y in zip(x, d1.values[:,0]):
        plt.text(x, y+0.1, y, ha='center', va='bottom')

    for x, y in temp:
        plt.text(x, -y-0.1, y, ha='center', va='bottom')
    for x, y in temp2:
        plt.text(x, 0, d1.index[x], ha='center', va='bottom', fontproperties=font_set)

    plt.xticks(())
    plt.yticks(())
    plt.title('%s买入卖出情况' % (bond_class), loc = 'left', fontproperties=font_set)
    plt.legend(['买入','卖出','净买入/卖出'])
    plt.savefig('C:/myproject/pyExcel/cfets_reports/%s买入卖出情况.png' % (bond_class))
    plt.show()

In [ ]:
by_bond_class(datas, "国债")
by_bond_class(datas, "中期票据")
by_bond_class(datas, "同业存单")